<a href="https://colab.research.google.com/github/meirbelo/gapevine_dataset/blob/main/officialModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from keras.optimizers import Adam

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)  # Rééchelonne les images

# Créer le générateur pour l'ensemble d'entraînement
train_generator = train_datagen.flow_from_directory(
    directory="/content/repartition/train",
    target_size=(224, 224),  # Taille des images après redimensionnement
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",  # Utilisez "categorical" pour plusieurs classes
    shuffle=True
)

# Créer le générateur pour l'ensemble de validation
validation_datagen = ImageDataGenerator(rescale=1./255)  # Rééchelonne les images

validation_generator = validation_datagen.flow_from_directory(
    directory="/content/repartition/validation",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",  # Utilisez "categorical" pour plusieurs classes
    shuffle=True
)

# Créer le générateur pour l'ensemble de test
test_datagen = ImageDataGenerator(rescale=1./255)  # Rééchelonne les images

test_generator = test_datagen.flow_from_directory(
    directory="/content/repartition/test",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",  # Utilisez "categorical" pour plusieurs classes
    shuffle=False
)


Found 14000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [ ]:
# Charger le modèle VGG16 pré-entraîné sur ImageNet et sans la dernière couche de classification
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Ajouter une nouvelle couche de classification binaire
x = base_model.output
x = Flatten()(x)  # Aplatir la sortie du modèle de base
x = Dense(512, activation='relu')(x)  # Ajouter une couche dense intermédiaire pour améliorer l'expressivité du modèle
x = Dropout(0.2)(x)  # Ajouter un Dropout pour réduire le surapprentissage
predictions = Dense(1, activation='sigmoid')(x)  # Ajouter une couche dense de sortie pour la classification binaire

# Définir le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# Congeler les couches du modèle de base pour ne pas les entraîner
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# Entraîner le modèle avec les générateurs d'images
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# Évaluer le modèle sur l'ensemble de test
scores = model.evaluate(
    test_generator,
    steps = test_generator.samples // test_generator.batch_size
)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Epoch 1/10
111/437 [======>.......................] - ETA: 1:46:54 - loss: 0.9590 - accuracy: 0.5000

In [ ]:
# Visualiser l'historique de l'entraînement
import matplotlib.pyplot as plt

# Afficher la courbe de la perte
plt.figure(figsize=(12, 4))
plt.plot(history.history['loss'], label='Perte (training data)')
plt.plot(history.history['val_loss'], label='Perte (validation data)')
plt.title('Perte pour la classification des feuilles de vigne')
plt.ylabel('Perte')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

# Afficher la courbe de la précision
plt.figure(figsize=(12, 4))
plt.plot(history.history['accuracy'], label='Précision (training data)')
plt.plot(history.history['val_accuracy'], label='Précision (validation data)')
plt.title('Précision pour la classification des feuilles de vigne')
plt.ylabel('Précision')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:

# Charger l'ensemble de données de test
test_path = "/content/repartition/test"
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    shuffle=False,
    seed=42,
)

# Effectuer les prédictions sur l'ensemble de données de test
predicted_labels = []
for images, _ in test_ds:
    predictions = efficientnetb0.predict(images)
    predicted_labels_batch = np.argmax(predictions, axis=1)
    predicted_labels.extend(predicted_labels_batch)

# Convertir les étiquettes prédites en noms de classes
predicted_labels = encoder.inverse_transform("/home/meir/Epitech/ia/voltron/repartition/test")

# Afficher les étiquettes prédites
print(predicted_labels)
